In [1]:
import os

In [2]:
%pwd

'd:\\project\\Face-to-BMI-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\Face-to-BMI-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    X_Train: Path
    y_Train: Path
    X_val: Path
    y_val: Path


In [6]:
from BMI.constants import *
from BMI.utils.common import read_yaml, create_directories

In [7]:
# import os
# from pathlib import Path
# from cnnClassifier.utils.common import read_yaml, create_directories
# from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
# from cnnClassifier.training_config import TrainingConfig  # Assuming you have a separate module for TrainingConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        # Define paths for training image and metadata
        X_train = os.path.join(
            self.config.data_ingestion.unzip_dir,"Train", 'X_train'
        )
        y_train = os.path.join(
            self.config.data_ingestion.unzip_dir, "Train", "y_train"
        )

        X_val = os.path.join(
            self.config.data_ingestion.unzip_dir,"Validation", 'X_val'
        )
        y_val = os.path.join(
            self.config.data_ingestion.unzip_dir, "Validation", "X_val"
        )

        # Create directories if they don't exist
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            X_Train= (X_train),
            y_Train= (y_train),
            X_val= (X_val),
            y_val= (y_val),
            
        )
        return training_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    



    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )



    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-09 00:33:49,966: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-09 00:33:49,972: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-09 00:33:49,974: INFO: common: created directory at: artifacts\training]
[2024-02-09 00:33:49,976: INFO: common: created directory at: artifacts\training]
[2024-02-09 00:33:49,978: INFO: common: created directory at: artifacts\prepare_base_model]


TypeError: TrainingConfig.__init__() got an unexpected keyword argument 'training_image'